# word embedding
#### 자연어를 처리할 때 count기반 문제
- 차원이 너무 커지는 문제
- 희소 차원임이 문제
- 단어의 의미의 차이를 알 수 없음.
- 거리를 계산할 수 있지만, (유사도를 비교할 순 있지만) 논리적인 의미를 알 수는 없음.


#### word embedding
- 단어간의 의미를 나타내고
- dense한 형태로 나타내보자
- 단어를 다차원 공간에 표현(의미를 갖는 벡터 공간에)

> 단어의 의미 자체를 다차원 공간에서 벡터화 필요
- 단어들 사이의 유사도 측정 가능
- 단어들간의 평균, 및 연산을 통해 추론 가능
- king - man = queen - woman => 단어간의 의미를 추론할 수 있음.!

> 벡터 계산
- va + vb = vc
- va - vb=>b에서 시작해서 a로 가는 벡터

# 방법
## 1.  Continous Bag of Words
- 주변 단어(문맥, context)들을 가지고 중간에 있는 단어(center word)들을 예측하는 방법
- The fat cat sat on the mat -> 중심단어: sat
- 중심 단어는 주변 단어들과 관련이 있기 때문에 
- 주변 단어: train data, 중심단어: y data 개념

 ### 1.1. 과정
 > 윈도우 슬라이딩을 통해 데이터셋 생성
 - 첫단어부터 중심단어로 설정해서, 주변 단어를 몇 개를 볼지 설정한 후, 주변 단어를 1로 놓고, 나머지는 0으로 설정
 - 단어를 옆으로 가면서, 주변단어들을 1로 설정하기
 - 한 문장내에서 윈도구 크기만큼 스캐닝을 하면서 training data 만들기
 - 중심단어: y , 주변단어: x data
 
 ### 1.2 네트워크 구성
 #### input -> projection layer -> pred(softmax) -> y라벨
 > input: 주변단어들(one hot) -> demension이 큼.
 - input shape(데이터를 column vector로 파악): 
 - 행-V(단어수), 열 - 주변단어를 onehot 인코딩한 결과
 
 > outout: 중심단어들(one hot)
  - y 라벨이 one hot이기 때문에 class의 개수가 매우 큼
 - V: 단어수
 
 > projection layer거치기(= 정사영)
 >> W: ( V 차원 -> M 차원으로 줄이기)
 - 실제로 신경망 알고리즘을 사용하지는 않음.
 - y= Wx 개념(b, 활성화 함수 개념 없음)
 - projectin의 개념:축 * 단위 벡터
 - 축: 단위벡터 * 벡터 = 벡터
 - 축: 단위벡터를 특정 방향으로 회전, 변환한 뒤 * 벡터
 - 특정 방향으로 단위 벡터를 곱하는 것: 프로젝션
 - x: 주변 단어
 - W: weight 매트릭스: (M, V)차원
 - M: 처음에 V였던 큰 디멘젼을 M 차원으로 바꾸고 싶을 때
 - 모든 축을 사용하지 않고, v개 만큼의 축이 있을 때 M개 만큼의 축을 선택하는 개념.(의미 없는 축 줄이기)
 
 >> 2차원 데이터-> 1차원 데이터로 줄이기
 - 2차원을 아무렇게나 1차원으로 줄이면 오류가 생기므로
 - (8, 10) 중 값을 1개만 사용하면 8 or 10 이므로 정보가 많이 소실됨.
 - 만약 (10, 8)이라는 데이터가 있다면 원점을 
 - 원점 이동 + 회전 이동하고 variance가 적은 축을 버림  -> 2차원 데이터를 1차원으로 변환가능=> 프로젝션
 - 모든 축을 사용하지 않고, v개 만큼의 축이 있을 때 M개 만큼의 축을 선택하는 개념.(의미 없는 축 줄이기)
 
 >> W': M차원-> 다시 V차원으로 늘리기
 - W'을 구하기
 - W' = V* M차원 -> err값을 계산하기 위해서만 사용하는 값. 실제로는 W만 사용
 - yhat = softmax(Z)
 - v차원으로 늘리는 이유: y라벨의 차원이 v이기 때문에
 - Z = 
 
 >> cross entropy
 - yhat과 y를 비교해서 loss를 최소화하면 W, W'를 학습시킴.
 - 실제로는 W만 사용하고, W'은 학습용도로만 사용
 
 
 

## 1.3. 실습

In [42]:
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import codecs
from konlpy.tag import Twitter
from gensim.models import word2vec
from konlpy.utils import pprint
from sklearn.manifold import TSNE

import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from tensorflow.keras.datasets import imdb
from konlpy.tag import Twitter
from konlpy.tag import Okt
from konlpy.tag import Kkma 
from konlpy.tag import Twitter
from pprint import pprint
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from wordcloud import WordCloud, STOPWORDS
from gensim import corpora, models
import numpy  as np
from PIL import Image
from wordcloud import ImageColorGenerator
import glob
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

In [3]:
sentences = [
                ['this', 'is', 'a',   'good',      'product'],
                ['it',   'is', 'a',   'excellent', 'product'],
                ['it',   'is', 'a',   'bad',       'product'],
                ['that', 'is', 'the', 'worst',     'product']
            ]

#### 모델

In [6]:
# 문장을 이용하여 단어와 벡터를 생성한다.-> W를 구함.
# 모델 만들기
model = Word2Vec(sentences,
                size = 20,  # vector의 차원
                window = 3, # 주변 단어를 몇개 볼것인가
                min_count = 1 # 최소 단어가 1번 나오는 것을 고려할 것이다
                )

#### 생성된 vector확인

- 단어별로 고정된 차원(feature가 생성)이 만들어졋으므로 
- 뒤에 이어서 모델을 만들 수 있음.

In [13]:
# 단어벡터를 구한다.
word_vectors = model.wv

In [14]:
# 'this'라는 단어의 벡터. 
# 원래 희소 행렬인데, size = 20차원으로 정했으므로, 20차원의 벡터로 변환됨.
word_vectors['this']

array([-0.02282666,  0.01874459,  0.0102811 ,  0.01935314,  0.00382596,
        0.01362558, -0.01492825, -0.01112475, -0.02164873, -0.00368071,
       -0.01426504, -0.00270334,  0.01467235,  0.0106786 ,  0.00282609,
       -0.02085758,  0.0165635 , -0.00653741,  0.02071085,  0.00148028],
      dtype=float32)

#### 어떤 단어들이 있는지 

In [24]:
vocabs = word_vectors.vocab.keys()
vocabs

dict_keys(['this', 'is', 'a', 'good', 'product', 'it', 'excellent', 'bad', 'that', 'the', 'worst'])

In [23]:
word_vectors.vocab.items()


dict_items([('this', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E02C8>), ('is', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0388>), ('a', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0408>), ('good', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0448>), ('product', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E04C8>), ('it', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0708>), ('excellent', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0748>), ('bad', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E07C8>), ('that', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0488>), ('the', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0608>), ('worst', <gensim.models.keyedvectors.Vocab object at 0x000001A56C0E0848>)])

#### 전체 단어 vector 확인하기

In [43]:
word_vectors_list = [word_vectors[v] for v in vocabs]
print(word_vectors_list)

[array([-0.02282666,  0.01874459,  0.0102811 ,  0.01935314,  0.00382596,
        0.01362558, -0.01492825, -0.01112475, -0.02164873, -0.00368071,
       -0.01426504, -0.00270334,  0.01467235,  0.0106786 ,  0.00282609,
       -0.02085758,  0.0165635 , -0.00653741,  0.02071085,  0.00148028],
      dtype=float32), array([ 0.00675884,  0.00153364, -0.00950851,  0.00809686, -0.00918463,
        0.01064638, -0.02291108,  0.00371675,  0.00851044, -0.00029214,
       -0.01142824, -0.0002227 , -0.0088951 , -0.00245749,  0.01111678,
       -0.0107454 , -0.01137813, -0.00131338, -0.00561119,  0.01871312],
      dtype=float32), array([-0.01787574,  0.0005055 ,  0.00707273,  0.00433004, -0.02444712,
        0.01246034, -0.01020078, -0.01451306, -0.01381973,  0.00220769,
       -0.01941462,  0.0105362 ,  0.00326828, -0.01761376,  0.01424268,
       -0.01527467,  0.00582997, -0.01815473,  0.00557335,  0.01154158],
      dtype=float32), array([ 0.01855993, -0.00199658,  0.0123276 , -0.02363487,  0.0002

### 1.3.1. 유사도 확인 : 코사인 distance로 확인

####  cos유사도: 각도로 확인
- 같으면 1 -> cos(0도 )= 1
- 다르면 0 -> cos(90도) = 0
- 결과 : -1 -> 반대 방향, cos(180) = -1


####  유클리안 거리와의 차이점
- 다른 두 벡터 안에 있는 두 점을 비교할 대(비례 관계의 삼각형)
- 유클리디안 거리로는 거리가 다르지만
- cos거리로는 같을 수 있음.->각도만 비교하기 때문

> cos
- 각도만 확인, 방향만 확인
- 정확하게 거리까지는 계산하지 않음.


In [35]:
model.wv.similarity('it', 'this')

0.14636609

#### 주어진 단어와 가장 유사한 단어

In [33]:
model.wv.most_similar('it', topn = 2)

[('a', 0.33982735872268677), ('bad', 0.25683528184890747)]

In [34]:
model.wv.most_similar('it')

[('a', 0.33982735872268677),
 ('bad', 0.25683528184890747),
 ('is', 0.1931551843881607),
 ('this', 0.14636608958244324),
 ('worst', 0.05984010919928551),
 ('that', -0.06858645379543304),
 ('product', -0.06895763427019119),
 ('the', -0.11150475591421127),
 ('excellent', -0.23884126543998718),
 ('good', -0.40791821479797363)]

## 1.4.  활용
#### 지금까지는 단어를 기준으로 한것.
#### 문장은 단어로 이루어져 있고, 우리는 문장에 관심이 있음
- 단어별로 embedding 하면 테마별로 같은 주제의 단어들은 관련된 단어끼리 뭉쳐있을 것.
- 임의의 문장이 주어졌을 때 평균을 구하면 어느 주제에 속했는지 알 수 있음.

# 2. 네이버 리뷰 분석

## 2.1. data
- 데이터는 문장들의 집합으로 구성
- 리스트 안에 리스트 형태로 구성

In [39]:
def read_data(filename):
    with open(filename, encoding = 'utf-8', mode = 'r') as f:
        data = [line.split('\t') for line in f.read() .splitlines()]
        data = data[1:] # header 제외
    return data

rating_train = read_data('ratings_train.txt')
        

In [40]:
rating_train[1]

['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1']

### 형태소 분석기로 단어 선별하기

In [44]:
okt = Okt()

In [50]:
# 형태소 분석해서 '단어/형태소 ' 형태로 만들기
def tokens(doc):
    return ['/'.join(t) for t in okt.pos(doc, norm = True, stem = True)]

In [51]:
tokens(rating_train[1][1])

['흠/Noun',
 '.../Punctuation',
 '포스터/Noun',
 '보고/Noun',
 '초딩/Noun',
 '영화/Noun',
 '줄/Noun',
 '..../Punctuation',
 '오버/Noun',
 '연기/Noun',
 '조차/Josa',
 '가볍다/Adjective',
 '않다/Verb']

In [54]:
docs = []
for row in rating_train:
    docs.append(row[1])
docs[:10]    

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.',
 '원작의 긴장감을 제대로 살려내지못했다.',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?']

In [86]:
len(docs)  # 15000차원

150000

In [53]:
data = [tokens(d) for d in docs]

In [55]:
data[0]

['아/Exclamation',
 '더빙/Noun',
 '../Punctuation',
 '진짜/Noun',
 '짜증나다/Adjective',
 '목소리/Noun']

## 2.2. model

In [81]:
w2v_model = Word2Vec(data,
                    size = 100, # 100차원
                    window = 3,
                    min_count = 3)

In [60]:
w2v_model.save('namver.model')

In [82]:
# 단어벡터를 구한다.
word_vectors = w2v_model.wv

## 2.3 총 단어 확인

In [84]:
len(word_vectors.vocab.keys())

21093

## 2.3. 유사도

In [72]:
# 남자 + 여배우 = 배우 = 여자
# 남자 - 배우 = 여자 - 여배우
# u' : 유니코드 인코딩 방식임.->안써도 됨..(editor가 unicode이므로 상관없음.)
w2v_model.wv.most_similar(positive = tokens(u'남자 여배우'),
                          negative = tokens(u'배우'),
                          topn = 5)

[('여자/Noun', 0.8384658098220825),
 ('할아버지/Noun', 0.7300465703010559),
 ('여자애/Noun', 0.7137463092803955),
 ('꼬마/Noun', 0.7094213962554932),
 ('아빠/Noun', 0.7092002630233765)]

In [69]:
w2v_model.wv.most_similar(tokens('정우성'))
                         

[('엄태웅/Noun', 0.8701173067092896),
 ('김희선/Noun', 0.8696517944335938),
 ('강지환/Noun', 0.8648170232772827),
 ('김윤석/Noun', 0.8604189157485962),
 ('박예진/Noun', 0.8580924272537231),
 ('최민식/Noun', 0.8521853685379028),
 ('패닝/Noun', 0.8500890731811523),
 ('노튼/Noun', 0.850029468536377),
 ('홍경인/Noun', 0.8488479852676392),
 ('강동원/Noun', 0.8487983345985413)]

In [74]:
w2v_model.wv.most_similar(tokens('토르'))


[('쇼타/Noun', 0.9126518964767456),
 ('야수/Noun', 0.9064512252807617),
 ('나토/Noun', 0.9046880006790161),
 ('화이트/Noun', 0.9034017324447632),
 ('칠이/Noun', 0.9013214111328125),
 ('썬더/Noun', 0.8994128704071045),
 ('해리/Noun', 0.8974124193191528),
 ('스타크/Noun', 0.8965331315994263),
 ('엠마/Noun', 0.8950753211975098),
 ('네스/Noun', 0.894395112991333)]